In [9]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Number of nodes
num_nodes = 20

# Create list of nodes
nodes = list(range(num_nodes))

# Define a function to simulate Hawkes process using Ogata's thinning algorithm
def simulate_hawkes_ogata(base_intensity, alpha, beta, max_time):
    events = []
    current_time = 0
    last_event_time = -np.inf
    lambda_star = base_intensity

    while current_time < max_time:
        # Propose next event time using exponential inter-event time
        u = np.random.uniform()
        current_time += -np.log(u) / lambda_star
        
        # Calculate conditional intensity at proposed time
        conditional_intensity = base_intensity + alpha * np.sum(np.exp(-beta * (current_time - np.array(events))))

        # Accept or reject the proposed event time
        if np.random.uniform() <= conditional_intensity / lambda_star:
            events.append(current_time)
            last_event_time = current_time
            lambda_star = conditional_intensity 
        else:
            lambda_star = conditional_intensity

    return events

# Maximum simulation time
max_time = 100

# Base intensity for the Hawkes process
base_intensity = 0.1

# Impact of each event
alpha = 0.5

# Decay rate
beta = 1.0

# Store events in a list
events = []

# Simulate the Hawkes process for each edge
for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        if i != j:
            times = simulate_hawkes_ogata(base_intensity, alpha, beta, max_time)
            for t in times:
                events.append((i, j, t))

# Sort events by time
events.sort(key=lambda x: x[2])

# Initialize adjacency matrix
adj_matrix = np.zeros((num_nodes, num_nodes))

# Store final events with event type in a list
final_events = []

# Process events and update adjacency matrix
for event in events:
    u, v, time = event
    # Simulate random update of the adjacency matrix just before the timestamp
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if adj_matrix[i, j] == 0 and np.random.rand() < 0.05:
                adj_matrix[i, j] = 1
                adj_matrix[j, i] = 1
                final_events.append((i, j, time, 0))  # Type 0 event

    # Mask off the events where A_ij = 0
    if adj_matrix[u, v] == 1:
        final_events.append((u, v, time, 1))  # Type 1 event

# Convert to DataFrame for better readability
final_events_df = pd.DataFrame(final_events, columns=['u', 'v', 'time', 'event_type'])

# Output the final events
final_events_df.to_csv('/Users/amberrrrrr/Desktop/huozhe/simulated_data/final_hawkes_process_events_hawkes.csv', index=False)
print("Final events have been generated and saved to 'final_hawkes_process_events_hawkes.csv'.")



Final events have been generated and saved to 'final_hawkes_process_events_hawkes.csv'.


In [10]:
final_events_df

,u,v,time,event_type
0,0,9,0.123536,0
1,0,10,0.123536,0
2,4,6,0.123536,0
3,4,9,0.123536,0
4,8,13,0.123536,0
...,...,...,...,...
2476,6,11,133.002779,1
2477,4,8,140.471800,1
2478,14,18,141.789402,1
2479,6,12,164.930100,1
